In [1]:
import requests
import re
import os
import json
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from dataclasses import dataclass
from collections import namedtuple, defaultdict

# 1. ОПИСАНИЕ

У Мосбиржи есть открытый интерфейс для получения различных рыночных данных. Он распологается по адресу http://iss.moex.com/iss/reference. Но к сожалению, его описание не идеально. Частично описано в документации https://fs.moex.com/files/6523. Часть поинтов недоступны без оформления подписки. Часть из них не работаеат, а их общее число заходит за 150 шт.

Большинству достаточно использовать несколько поинтов - получения истории торгов и описания инструментов.
Для качественной аналитики этих данных может быть недостаточно. А для некоторых задач, этого критически недостаточно (например, в выялвении фрода, инсайда и манипулирования на финансовых рынках). Приведем часть интересных кейсов:

1. Инсайдеры могут заблоговременно совершать сделки перед раскрытием существенных факторов;
2. Неликивидные бумаги посредством сделок с договоренностью могут переводить в ликвидные с целью соответствия правилам инвест. деклараций, привлечения публики, массовых скупок/распродаж, завышения залога по сделкам РЕПО и прочего (и вплоть до завышения/занижения цены при SPO/обратных выкупах);
3. Часть бумаг (как правило неликвдиных бондов) могут целенаправленно завышать в цене, с целью извлечения дополнительных комиссионных от объема/доходности средств в управлении;
4. Обход законодательных ограничений или лимитов участника торгов (например, продажа ФИ внутри одной УК);
5. Подгод рыночных цен под будущую крупную внебиржевую сделку (вне основных торгов). Объемы таких сделок могут значительно завышать обороты рынка данной бумаги;
6. Влияние на цены фиксинга связанных ФИ/ПФИ/курсов валют. Крупные остаки ПФИ могут стимулировать к совершению сделок искажающих цены на спот рынке (или наоборот, но реже); 
7. Искусственное искажение спроса и предложения, злоупотребление правами маркет-мейкера;
8. Изменение объемов свободной ликвидности (резкие обвалы на рынке часто начинаются с недостатка ликвидности: распрадажа активов для покрытия текущих обязательств). Поэтому и нужно постоянно мониторить здоровье финансовой сферы;
9. Внутренние мошенничества: совершение опережающих сделок перед крупными заявками, нарушение приниципа китайских стен;
10. Подковерные игры бенефициров.
11. Непрофессиональное исполнение поручений и т.п. 

Это далеко не весь список кейсов. Но уже из этого следует, что нужно следить не только за историей основных торгов, но и за показателями других площадок, ПФИ, РЕПО, фиксингом, ставками, валютой, вложениями фондов, новостями и т.д.

Лучше данные - это история логов заявок, но, к сожалению они доступны только при оформлении подписки.


**Весь последующий анализ проводится для разработки библиотеки для удобной работы с апи Мосбиржи**

In [2]:
root_url = "http://iss.moex.com"
docs_url = "/iss/reference"

In [3]:
req = requests.get(root_url + docs_url)
soup = bs(req.content).body.dl

In [4]:
@dataclass
class MoexApi:
    url_faq: str
    url_template: str
    description: str = ''

# 2. GET API MOEX

In [5]:
# GET API
FINAL_DATA = []
tmp_list = []
for el in soup.next_elements:
    if el.name == "dt":  # href in child element "a"
        a = el.contents[0]
        tmp_list = [
            root_url + docs_url + a.attrs['href'].replace('.', ''),
            root_url + a.text
        ]
    
    elif el.name == "p":  # Description
        tmp_list.append(el.text.strip().replace('\n', ''))
        if len(tmp_list) == 3:
            FINAL_DATA.append(
                MoexApi(url_faq=tmp_list[0], url_template=tmp_list[1], description=tmp_list[2])
            )
            tmp_list = []
        else:
            print(tmp_list)
            raise Exception("SOMETHING WENT WRONG")

In [6]:
print('Quantity of endpoint =', len(FINAL_DATA))
FINAL_DATA[:5]

Quantity of endpoint = 151


[MoexApi(url_faq='http://iss.moex.com/iss/reference/5', url_template='http://iss.moex.com/iss/securities', description='Список бумаг торгуемых на московской бирже.'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/13', url_template='http://iss.moex.com/iss/securities/[security]', description='Получить спецификацию инструмента. Например: https://iss.moex.com/iss/securities/IMOEX.xml'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/160', url_template='http://iss.moex.com/iss/securities/[security]/indices', description='Список индексов в которые входит бумага'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/214', url_template='http://iss.moex.com/iss/securities/[security]/aggregates', description='Агрегированные итоги торгов за дату по рынкам'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/823', url_template='http://iss.moex.com/iss/engines/[engine]/markets/[market]/secstats', description='Промежуточные "Итоги дня".Только для фондового рынка')]

In [7]:
# GET ENTITIES (GLOBALS)
ENTITIES = defaultdict(list)

for num, api in enumerate(FINAL_DATA):
    entities = list(map(lambda entity: entity[1:-1], re.findall(r'\[\w*\]', api.url_template)))
    
    if entities:
        api.url_template = api.url_template.replace('[', '{').replace(']', '}')
        for entity in entities:
            ENTITIES[entity].append(num)

# 3. CHECK Entities

In [8]:
# ✅❌
CHECK_ENTITY = {entity: '❌' for entity in sorted(ENTITIES.keys())}
ENTITY_DESCRIPTIONS = {}
CHECK_ENTITY

{'asset': '❌',
 'board': '❌',
 'boardgroup': '❌',
 'collection': '❌',
 'datatype': '❌',
 'engine': '❌',
 'event_id': '❌',
 'indexid': '❌',
 'market': '❌',
 'news_id': '❌',
 'period': '❌',
 'report_name': '❌',
 'securities': '❌',
 'security': '❌',
 'securitygroup': '❌',
 'session': '❌',
 'ticker': '❌',
 'year': '❌'}

In [9]:
# NEXT CELLS - Analitics
[FINAL_DATA[inx] for inx in ENTITIES['asset']]

[MoexApi(url_faq='http://iss.moex.com/iss/reference/877', url_template='http://iss.moex.com/iss/statistics/engines/futures/markets/options/assets/{asset}', description='Опционные серии'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/879', url_template='http://iss.moex.com/iss/statistics/engines/futures/markets/options/assets/{asset}/volumes', description='Объем торгов для опционной серии'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/881', url_template='http://iss.moex.com/iss/statistics/engines/futures/markets/options/assets/{asset}/optionboard', description='Доска опционов'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/883', url_template='http://iss.moex.com/iss/statistics/engines/futures/markets/options/assets/{asset}/openpositions', description='Открытые позиции по опционной серии'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/885', url_template='http://iss.moex.com/iss/statistics/engines/futures/markets/options/assets/{asset}/turnovers', description=

In [10]:
ENTITY_DESCRIPTIONS['asset'] = "Базовый актив опциона"  # Требуется анализ. Скорее всего БА в справочниках опционов

In [11]:
# Получить глобальные справочники ISS. Например: https://iss.moex.com/iss/index.xml
global_entity = requests.get("https://iss.moex.com/iss/index.json").json()
global_entity.keys()

dict_keys(['engines', 'markets', 'boards', 'boardgroups', 'durations', 'securitytypes', 'securitygroups', 'securitycollections'])

In [12]:
for key, val in global_entity.items():
    print(key, '\n',  val['columns'], '\n')

engines 
 ['id', 'name', 'title'] 

markets 
 ['id', 'trade_engine_id', 'trade_engine_name', 'trade_engine_title', 'market_name', 'market_title', 'market_id', 'marketplace', 'is_otc', 'has_history_files', 'has_history_trades_files', 'has_trades', 'has_history', 'has_candles', 'has_orderbook', 'has_tradingsession', 'has_extra_yields', 'has_delay'] 

boards 
 ['id', 'board_group_id', 'engine_id', 'market_id', 'boardid', 'board_title', 'is_traded', 'has_candles', 'is_primary'] 

boardgroups 
 ['id', 'trade_engine_id', 'trade_engine_name', 'trade_engine_title', 'market_id', 'market_name', 'name', 'title', 'is_default', 'board_group_id', 'is_traded', 'is_order_driven', 'category'] 

durations 
 ['interval', 'duration', 'days', 'title', 'hint'] 

securitytypes 
 ['id', 'trade_engine_id', 'trade_engine_name', 'trade_engine_title', 'security_type_name', 'security_type_title', 'security_group_name', 'stock_type'] 

securitygroups 
 ['id', 'name', 'title', 'is_hidden'] 

securitycollections 
 ['

In [13]:
class Globals:
    __instance = None
    globals_url = "https://iss.moex.com/iss/index.json"
    url_index_id = "https://iss.moex.com/iss/statistics/engines/stock/markets/index/analytics.json"
    
    base_attr = namedtuple("base_attr", "main description")
    
    main_attribute = {
        "engines": base_attr("name", "title"),
        "markets": base_attr("market_name", "market_title"),
        "boards": base_attr("boardid", "board_title"),
        "boardgroups": base_attr("name", "title"),
        "durations": base_attr("interval", "title"),
        "securitytypes": base_attr("security_type_name", "security_type_title"),
        "securitygroups": base_attr("name", "title"),
        "securitycollections": base_attr("name", "title"),
    }
    datatypes = ["securities", "trades"]
    
    report_names = pd.DataFrame(
        data=[
            [
                "numtrades", 
                "Информация о количестве договоров по инструментам, "
                "являющимся производными финансовыми инструментами (по валютным парам)",
            ],
            [
                "participants",
                "Информация о количестве лиц, имеющих открытые позиции по инструментам, "
                "являющимся производными финансовыми инструментами (по валютным парам)",
            ],
            [
                "openpositions",
                "Информация об открытых позициях по инструментам, являющимся производными "
                "финансовыми инструментами (по валютным парам)",
            ],
            [
                "expirationparticipants",
                "Информация о количестве лиц, имеющих открытые позиции по договорам, "
                "являющимся производными финансовыми инструментами (по срокам экспирации)",
            ],
            [
                "expirationopenpositions",
                "Информация об объеме открытых позиций по договорам, являющимся производными "
                "финансовыми инструментами (по срокам экспирации)",
            ],
        ],
        columns=['report_name', 'decription']
    )
    report_names.set_index('report_name', inplace=True)
    
    sessions = pd.DataFrame(
        data=[
            [1, "Основная сессия"], [2, "Вечерняя сессия"], [3, "Итого (все сессии)"], [0, "Утренняя сессия"]
        ], 
        columns=['sessions', 'decription']
    )
    sessions.set_index('sessions', inplace=True)
    
    
    def __new__(cls, *args, **kwargs):
        if not cls.__instance:                
            cls.__instance = super().__new__(cls, *args, **kwargs)
        return cls.__instance
    
    def __init__(self):
        moex_dict = requests.get(self.globals_url).json()
        for entity, columns in self.main_attribute.items():
            setattr(self, entity, pd.DataFrame(
                data=moex_dict[entity]['data'],
                columns=moex_dict[entity]['columns'],
                ).set_index(columns.main)
            )
        self.indexids = self.get_index_id()
    
    def get_index_id(self):
        index_ids = requests.get(self.url_index_id).json()
        df_index_ids = pd.DataFrame(
            data=index_ids['indices']['data'], columns=index_ids['indices']['columns']
        )
        df_index_ids.set_index('indexid', inplace=True)
        return df_index_ids
    
    
    def description(self, entity: str, name):
        entities = entity + "s"
        use_entity = getattr(self, entities, None)
        if use_entity is None:
            raise NameError(f"Entity '{entity}' is not found!")
        if not isinstance(use_entity, pd.DataFrame):
            raise ValueError(f"Entity '{entity}' hasn't description")
        return getattr(self, entity + "s").loc[name, (self.main_attribute[entities].description,)]
            
    
GLOBAL_GUID = Globals()
GLOBAL_GUID.markets

,id,trade_engine_id,trade_engine_name,trade_engine_title,market_title,market_id,marketplace,is_otc,has_history_files,has_history_trades_files,has_trades,has_history,has_candles,has_orderbook,has_tradingsession,has_extra_yields,has_delay
market_name,,,,,,,,,,,,,,,,,
index,5,1,stock,Фондовый рынок и рынок депозитов,Индексы фондового рынка,5,INDICES,0,1.0,0.0,1.0,1.0,1,0.0,1,0,0
shares,1,1,stock,Фондовый рынок и рынок депозитов,Рынок акций,1,MXSE,0,1.0,1.0,1.0,1.0,1,1.0,1,0,1
bonds,2,1,stock,Фондовый рынок и рынок депозитов,Рынок облигаций,2,MXSE,0,1.0,1.0,1.0,1.0,1,1.0,1,1,1
ndm,4,1,stock,Фондовый рынок и рынок депозитов,Режим переговорных сделок,4,None,0,1.0,1.0,1.0,1.0,0,0.0,1,1,1
otc,29,1,stock,Фондовый рынок и рынок депозитов,ОТС,29,None,0,0.0,0.0,0.0,1.0,0,0.0,0,0,0
ccp,27,1,stock,Фондовый рынок и рынок депозитов,РЕПО с ЦК,27,MXSE,0,1.0,1.0,1.0,1.0,1,1.0,1,0,1
deposit,35,1,stock,Фондовый рынок и рынок депозитов,Депозиты с ЦК,35,None,0,1.0,0.0,0.0,1.0,0,0.0,0,0,0
repo,3,1,stock,Фондовый рынок и рынок депозитов,Рынок сделок РЕПО,3,None,0,1.0,1.0,1.0,1.0,0,0.0,1,0,1
qnv,28,1,stock,Фондовый рынок и рынок депозитов,Квал. инвесторы,28,None,0,1.0,1.0,1.0,1.0,1,0.0,0,0,1


In [14]:
GLOBAL_GUID.description("board", "TQBS")

'Т+: А2-Акции и паи - безадрес.'

# 4. Анализ сущностей

In [15]:
for entity in Globals.main_attribute:
    entity = entity[:-1]
    if entity in CHECK_ENTITY:
        CHECK_ENTITY[entity] = '✅'
    else:
        print(entity)
CHECK_ENTITY

duration
securitytype
securitycollection


{'asset': '❌',
 'board': '✅',
 'boardgroup': '✅',
 'collection': '❌',
 'datatype': '❌',
 'engine': '✅',
 'event_id': '❌',
 'indexid': '❌',
 'market': '✅',
 'news_id': '❌',
 'period': '❌',
 'report_name': '❌',
 'securities': '❌',
 'security': '❌',
 'securitygroup': '✅',
 'session': '❌',
 'ticker': '❌',
 'year': '❌'}

In [16]:
ENTITY_DESCRIPTIONS["board"] = "Код торговой площадки (режим торгов)"
ENTITY_DESCRIPTIONS["boardgroup"] = "Группа торговых площадок"
ENTITY_DESCRIPTIONS["engine"] = "Признак рынка (фондовый, валютный, срочный...)"
ENTITY_DESCRIPTIONS["market"] = "Классфикация рынка: спот, индексы, рпс, репо, размещение, валюта и т.п."
ENTITY_DESCRIPTIONS["securitygroup"] = "Класс финансового инструмента"

In [17]:
# collection
[FINAL_DATA[inx] for inx in ENTITIES['collection']]

[MoexApi(url_faq='http://iss.moex.com/iss/reference/130', url_template='http://iss.moex.com/iss/securitygroups/{securitygroup}/collections/{collection}', description='Коллекция ценных бумаг входящие в группу'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/131', url_template='http://iss.moex.com/iss/securitygroups/{securitygroup}/collections/{collection}/securities', description='Описание инструментов')]

In [18]:
# Сollections == securitycollections
ENTITY_DESCRIPTIONS["collection"] = (
    "Группировка финансовых инструментов по признакам (уровень, тип и тд). В словаре используется как securitycollection"
)
CHECK_ENTITY["collection"] = '✅'

In [19]:
# collection
[FINAL_DATA[inx] for inx in ENTITIES['datatype']]  # datatype может принимать значения securities или trades.

[MoexApi(url_faq='http://iss.moex.com/iss/reference/114', url_template='http://iss.moex.com/iss/archives/engines/{engine}/markets/{market}/{datatype}/years', description='Список годов, за которые существуют ссылки на файлы с архивом сделок и исторической биржевой информацией.\rdatatype может принимать значения securities или trades.'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/116', url_template='http://iss.moex.com/iss/archives/engines/{engine}/markets/{market}/{datatype}/{period}', description='Получить список ccылок на годовые/месячные/дневные файлы с архивом сделок и исторической биржевой информацией.\rdatatype может принимать значения securities или trades. \rperiod может принимать значения yearly, monthly или daily.\rПомесячные данные доступны только за последние 30 дней.'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/115', url_template='http://iss.moex.com/iss/archives/engines/{engine}/markets/{market}/{datatype}/years/{year}/months', description='Список месяце

#### *Add dict of datatype directly in class Globals (above)*

In [20]:
ENTITY_DESCRIPTIONS["datatype"] = "securities or trades"
CHECK_ENTITY["datatype"] = '✅'

In [21]:
# event_id
[FINAL_DATA[inx] for inx in ENTITIES['event_id']]

[MoexApi(url_faq='http://iss.moex.com/iss/reference/194', url_template='http://iss.moex.com/iss/events/{event_id}', description='Контент мероприятия биржи')]

In [22]:
CHECK_ENTITY["event_id"] = '✅'
ENTITY_DESCRIPTIONS["event_id"] = (
    "ID мероприятия проводимого Мосбиржей. Идут в порядке возрастания. Только ближайшие события "
    "можно посмотреть на /iss/events"
)

In [23]:
# indexid
[FINAL_DATA[inx] for inx in ENTITIES['indexid']]

[MoexApi(url_faq='http://iss.moex.com/iss/reference/147', url_template='http://iss.moex.com/iss/statistics/engines/stock/markets/index/analytics/{indexid}', description='Аналитические показатели за дату'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/148', url_template='http://iss.moex.com/iss/statistics/engines/stock/markets/index/analytics/{indexid}/tickers', description='Список тикеров за все время торгов'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/149', url_template='http://iss.moex.com/iss/statistics/engines/stock/markets/index/analytics/{indexid}/tickers/{ticker}', description='Информация по тикеру')]

In [24]:
# def get_url_index():
url_index_id = "https://iss.moex.com/iss/statistics/engines/stock/markets/index/analytics.json"
index_ids = requests.get(url_index_id).json()
df_index_ids = pd.DataFrame(
    data=index_ids['indices']['data'], columns=index_ids['indices']['columns']
)
df_index_ids.set_index('indexid', inplace=True)
df_index_ids

,shortname,from,till
indexid,,,
IMOEX,Индекс МосБиржи,2001-01-03,2023-05-17
MOEX10,Индекс МосБиржи 10,2008-05-27,2023-05-17
RTSI,Индекс РТС,2009-09-30,2023-05-17
MOEXBC,Индекс голубых фишек,2010-01-14,2023-05-17
MOEXBMI,Индекс широкого рынка,2011-12-30,2023-05-17
...,...,...,...
RUEU10,MOEX Еврооблигации 10,2018-12-27,2021-10-12
RUEUESG,RUEUESG,2021-03-21,2021-10-12
RUEURAIF,Индекса МосБиржи еврооблигаций RUEURAIF,2020-06-01,2021-10-12


#### Add dict of indexid directly in class Globals (above)

In [25]:
CHECK_ENTITY["indexid"] = '✅'
ENTITY_DESCRIPTIONS["indexid"] = "Код индекса (бенчмарка) публикуемого и/или расчитываемого Мосбиржей"

In [26]:
# news_id
[FINAL_DATA[inx] for inx in ENTITIES['news_id']] 

[MoexApi(url_faq='http://iss.moex.com/iss/reference/192', url_template='http://iss.moex.com/iss/sitenews/{news_id}', description='Новость сайта')]

In [27]:
CHECK_ENTITY["news_id"] = '✅'
ENTITY_DESCRIPTIONS["news_id"] = "Код новостной рассылки Мосбиржи. Реестр новостей можно увидить на /iss/sitenews"

In [28]:
# period
[FINAL_DATA[inx] for inx in ENTITIES['period']] # period может принимать значения yearly, monthly или daily

[MoexApi(url_faq='http://iss.moex.com/iss/reference/116', url_template='http://iss.moex.com/iss/archives/engines/{engine}/markets/{market}/{datatype}/{period}', description='Получить список ccылок на годовые/месячные/дневные файлы с архивом сделок и исторической биржевой информацией.\rdatatype может принимать значения securities или trades. \rperiod может принимать значения yearly, monthly или daily.\rПомесячные данные доступны только за последние 30 дней.')]

In [29]:
CHECK_ENTITY["period"] = '✅'
ENTITY_DESCRIPTIONS["period"] = "yearly, monthly или daily"

In [30]:
# report_name
[FINAL_DATA[inx] for inx in ENTITIES['report_name']]

[MoexApi(url_faq='http://iss.moex.com/iss/reference/219', url_template='http://iss.moex.com/iss/statistics/engines/{engine}/derivatives/{report_name}', description='Еженедельные отчеты по валютным деривативам:\r\rnumtrades - Информация о количестве договоров по инструментам, являющимся производными финансовыми инструментами (по валютным парам)\rparticipants - Информация о количестве лиц, имеющих открытые позиции по инструментам, являющимся производными финансовыми инструментами (по валютным парам)\ropenpositions - Информация об открытых позициях по инструментам, являющимся производными финансовыми инструментами (по валютным парам)\rexpirationparticipants - Информация о количестве лиц, имеющих открытые позиции по договорам, являющимся производными финансовыми инструментами (по срокам экспирации)\rexpirationopenpositions - Информация об объеме открытых позиций по договорам, являющимся производными финансовыми инструментами (по срокам экспирации)'),
 MoexApi(url_faq='http://iss.moex.com/i

In [31]:
data_desc_reportname = [
    [
        "numtrades", 
        "Информация о количестве договоров по инструментам, "
        "являющимся производными финансовыми инструментами (по валютным парам)",
    ],
    [
        "participants",
        "Информация о количестве лиц, имеющих открытые позиции по инструментам, "
        "являющимся производными финансовыми инструментами (по валютным парам)",
    ],
    [
        "openpositions",
        "Информация об открытых позициях по инструментам, являющимся производными "
        "финансовыми инструментами (по валютным парам)",
    ],
    [
        "expirationparticipants",
        "Информация о количестве лиц, имеющих открытые позиции по договорам, "
        "являющимся производными финансовыми инструментами (по срокам экспирации)",
    ],
    [
        "expirationopenpositions",
        "Информация об объеме открытых позиций по договорам, являющимся производными "
        "финансовыми инструментами (по срокам экспирации)",
    ],
]
df_report_name = pd.DataFrame(data=data_desc_reportname, columns=['report_name', 'decription'])
df_report_name

,report_name,decription
0,numtrades,Информация о количестве договоров по инструмен...
1,participants,"Информация о количестве лиц, имеющих открытые ..."
2,openpositions,Информация об открытых позициях по инструмента...
3,expirationparticipants,"Информация о количестве лиц, имеющих открытые ..."
4,expirationopenpositions,Информация об объеме открытых позиций по догов...


#### *Add data of report_name directly in class Globals (above)*

In [32]:
CHECK_ENTITY["report_name"] = '✅'
ENTITY_DESCRIPTIONS["report_name"] = (
    "Еженедельные отчеты по ВАЛЮТНЫМ (! только !) деривативам. По первичному анализу - OTC / c ЦК и нестандартные ПФИ. Нужны детали."
)

In [33]:
[FINAL_DATA[inx] for inx in ENTITIES['securities']]

[MoexApi(url_faq='http://iss.moex.com/iss/reference/789', url_template='http://iss.moex.com/iss/history/engines/{engine}/markets/{market}/.*?{securities}/columns', description='Получить описание полей для запросов исторических данных по бумагам для рынка.')]

Данный поинт возращает тоже, что и /iss/engines/[engine]/markets/[market]/.*?securities/columns


Обращение к /iss/engines/[engine]/markets/[market]/securities/columns вернет описание исторических данных. Адрес выше можно исключить из доступных апи (хотя в истории поля могли меняться)



In [34]:
CHECK_ENTITY.pop("securities")

'❌'

In [35]:
[FINAL_DATA[inx] for inx in ENTITIES['security']][:10]

[MoexApi(url_faq='http://iss.moex.com/iss/reference/13', url_template='http://iss.moex.com/iss/securities/{security}', description='Получить спецификацию инструмента. Например: https://iss.moex.com/iss/securities/IMOEX.xml'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/160', url_template='http://iss.moex.com/iss/securities/{security}/indices', description='Список индексов в которые входит бумага'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/214', url_template='http://iss.moex.com/iss/securities/{security}/aggregates', description='Агрегированные итоги торгов за дату по рынкам'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/817', url_template='http://iss.moex.com/iss/history/engines/{engine}/markets/{market}/sessions/{session}/securities/{security}', description='Получить историю по одной бумаге на рынке за интервал дат.'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/819', url_template='http://iss.moex.com/iss/history/engines/{engine}/markets/{market}/ses

Под security используется secid ФИ на Мосбирже. Получение торгуемых инструментов в разрезе торгуемости, площадок, а также их описание - отдельная задача, которую решим позже. Сейчас отметим сущность как прочеканную.

In [36]:
CHECK_ENTITY["security"] = '✅'
ENTITY_DESCRIPTIONS["security"] = "Secid финансового инструмента"

In [37]:
[FINAL_DATA[inx] for inx in ENTITIES['session']]

[MoexApi(url_faq='http://iss.moex.com/iss/reference/813', url_template='http://iss.moex.com/iss/history/engines/{engine}/markets/{market}/sessions/{session}/securities', description='Получить историю по всем бумагам на рынке за одну дату.Например: https://iss.moex.com/iss/history/engines/stock/markets/index/securities.xml?date=2010-11-22'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/817', url_template='http://iss.moex.com/iss/history/engines/{engine}/markets/{market}/sessions/{session}/securities/{security}', description='Получить историю по одной бумаге на рынке за интервал дат.'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/825', url_template='http://iss.moex.com/iss/history/engines/{engine}/markets/{market}/session/{session}/boardgroups/{boardgroup}/securities', description='Получить историю торгов для всех бумаг на указанной группе режимов торгов за указанную дату.'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/819', url_template='http://iss.moex.com/iss/hi

Cессии:
- 0 - Утренняя
- 1 - Основная
- 2 - Вечерняя
- 3 - Итого

#### Add dict of session directly in class Globals (above)

In [38]:
CHECK_ENTITY["session"] = '✅'
ENTITY_DESCRIPTIONS["session"] = "Сессия торгов (утро, вечер, основная, итого)"
GLOBAL_GUID.sessions

,decription
sessions,
1,Основная сессия
2,Вечерняя сессия
3,Итого (все сессии)
0,Утренняя сессия


In [39]:
[FINAL_DATA[inx] for inx in ENTITIES['ticker']]

[MoexApi(url_faq='http://iss.moex.com/iss/reference/149', url_template='http://iss.moex.com/iss/statistics/engines/stock/markets/index/analytics/{indexid}/tickers/{ticker}', description='Информация по тикеру')]

Не получилось добыть данные. 

/iss/statistics/engines/stock/markets/index/analytics/[indexid]/tickers возвращает список тикеров формирующих индекс ([indexid]). Но добавление тикера в запрос возвращает пустой результат. Возможно работает с ограниченным числом индексов 
или поинт доступен только для платных подписчиков.
По полям - капитализация эмитента, вляиние бумаги на индекс и прочие статистические показатели.
Также ticker это secid ФИ или сущность - security.

Проставим как прочеканную.

In [40]:
ENTITY_DESCRIPTIONS["ticker"] = "Тикер бумаги в индексе долевых ФИ. По сути - Secid. Детали: /iss/statistics/engines/stock/markets/index/analytics/[indexid]/tickers"
CHECK_ENTITY["ticker"] = '✅'

In [41]:
[FINAL_DATA[inx] for inx in ENTITIES['year']]

[MoexApi(url_faq='http://iss.moex.com/iss/reference/115', url_template='http://iss.moex.com/iss/archives/engines/{engine}/markets/{market}/{datatype}/years/{year}/months', description='Список месяцев в году, за которые существуют ссылки на файлы с архивом сделок и исторической биржевой информацией.\rdatatype может принимать значения securities или trades.')]

In [42]:
# Год - просто год:)
ENTITY_DESCRIPTIONS["year"] = "Год, только для архивов"
CHECK_ENTITY["year"] = '✅'

In [43]:
# asset
[FINAL_DATA[inx] for inx in ENTITIES['asset']]

[MoexApi(url_faq='http://iss.moex.com/iss/reference/877', url_template='http://iss.moex.com/iss/statistics/engines/futures/markets/options/assets/{asset}', description='Опционные серии'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/879', url_template='http://iss.moex.com/iss/statistics/engines/futures/markets/options/assets/{asset}/volumes', description='Объем торгов для опционной серии'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/881', url_template='http://iss.moex.com/iss/statistics/engines/futures/markets/options/assets/{asset}/optionboard', description='Доска опционов'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/883', url_template='http://iss.moex.com/iss/statistics/engines/futures/markets/options/assets/{asset}/openpositions', description='Открытые позиции по опционной серии'),
 MoexApi(url_faq='http://iss.moex.com/iss/reference/885', url_template='http://iss.moex.com/iss/statistics/engines/futures/markets/options/assets/{asset}/turnovers', description=

По /iss/securitygroups/[securitygroup]/collections/[collection]/securities можно получить описание опцинов. Полученные поля содержат **ASSETCODE** - secid базового ФИ. Поэтому поле asset считаем прочеканым.

In [44]:
ENTITY_DESCRIPTIONS["asset"] = "Secid базового актива для опционов."
CHECK_ENTITY["asset"] = '✅'

In [45]:
CHECK_ENTITY

{'asset': '✅',
 'board': '✅',
 'boardgroup': '✅',
 'collection': '✅',
 'datatype': '✅',
 'engine': '✅',
 'event_id': '✅',
 'indexid': '✅',
 'market': '✅',
 'news_id': '✅',
 'period': '✅',
 'report_name': '✅',
 'security': '✅',
 'securitygroup': '✅',
 'session': '✅',
 'ticker': '✅',
 'year': '✅'}

In [46]:
ENTITY_DESCRIPTIONS

{'asset': 'Secid базового актива для опционов.',
 'board': 'Код торговой площадки (режим торгов)',
 'boardgroup': 'Группа торговых площадок',
 'engine': 'Признак рынка (фондовый, валютный, срочный...)',
 'market': 'Классфикация рынка: спот, индексы, рпс, репо, размещение, валюта и т.п.',
 'securitygroup': 'Класс финансового инструмента',
 'collection': 'Группировка финансовых инструментов по признакам (уровень, тип и тд). В словаре используется как securitycollection',
 'datatype': 'securities or trades',
 'event_id': 'ID мероприятия проводимого Мосбиржей. Идут в порядке возрастания. Только ближайшие события можно посмотреть на /iss/events',
 'indexid': 'Код индекса (бенчмарка) публикуемого и/или расчитываемого Мосбиржей',
 'news_id': 'Код новостной рассылки Мосбиржи. Реестр новостей можно увидить на /iss/sitenews',
 'period': 'yearly, monthly или daily',
 'report_name': 'Еженедельные отчеты по ВАЛЮТНЫМ (! только !) деривативам. По первичному анализу - OTC / c ЦК и нестандартные ПФИ. Н

# 5. Доступность endpoint'ов


Так как некоторые эндпоинты ответственны за определенный тип ФИ, рынка, площадки и тд, создадим несколько базовых словарей, данные которых будут подставляться в url при получении пустых ответов.
В будущем, можно решить задачу связи сущностей друг с другом.

In [47]:
# Начнем с сущностей:
url_params = [{"engine": engine} for engine in GLOBAL_GUID.engines.index]
state = url_params.pop(1)
url_params.insert(3, state)
url_params

[{'engine': 'stock'},
 {'engine': 'currency'},
 {'engine': 'futures'},
 {'engine': 'state'},
 {'engine': 'commodity'},
 {'engine': 'interventions'},
 {'engine': 'offboard'},
 {'engine': 'agro'},
 {'engine': 'otc'}]

In [48]:
# Добавим первичные рынки по каждой сущности (есть маркетплайс и есть книга заявок)
for dict_ in url_params:
    engine = dict_['engine']
    market_slice = GLOBAL_GUID.markets.query('trade_engine_name==@engine')
    if market_slice.shape[0] == 1:
        dict_['market'] = market_slice.iloc[0].name
    else:
        market_slice_tmp = market_slice.query("marketplace.notnull()")
        if market_slice_tmp.shape[0]:
            dict_['market'] = market_slice_tmp.sort_values(by=['id']).iloc[0].name
        else:
            dict_['market'] = market_slice.query("has_orderbook==1").sort_values(by=['id']).iloc[0].name
url_params

[{'engine': 'stock', 'market': 'shares'},
 {'engine': 'currency', 'market': 'selt'},
 {'engine': 'futures', 'market': 'forts'},
 {'engine': 'state', 'market': 'bonds'},
 {'engine': 'commodity', 'market': 'futures'},
 {'engine': 'interventions', 'market': 'grain'},
 {'engine': 'offboard', 'market': 'bonds'},
 {'engine': 'agro', 'market': 'sugar'},
 {'engine': 'otc', 'market': 'bonds'}]

In [49]:
# Добавим первичную группу площадок по каждой сущности
for dict_ in url_params:
    engine, market = dict_['engine'], dict_['market']
    dict_['boardgroup'] = GLOBAL_GUID.boardgroups.query(
        'market_name==@market and trade_engine_name==@engine and is_default==1'
    ).sort_values(by=['id']).iloc[0].name
url_params

[{'engine': 'stock', 'market': 'shares', 'boardgroup': 'stock_shares_tplus'},
 {'engine': 'currency', 'market': 'selt', 'boardgroup': 'currency'},
 {'engine': 'futures', 'market': 'forts', 'boardgroup': 'futures_forts'},
 {'engine': 'state', 'market': 'bonds', 'boardgroup': 'state'},
 {'engine': 'commodity',
  'market': 'futures',
  'boardgroup': 'commodity_futures'},
 {'engine': 'interventions',
  'market': 'grain',
  'boardgroup': 'interventions_grain'},
 {'engine': 'offboard', 'market': 'bonds', 'boardgroup': 'offboard_bond_all'},
 {'engine': 'agro', 'market': 'sugar', 'boardgroup': 'agro_sugar_all'},
 {'engine': 'otc', 'market': 'bonds', 'boardgroup': 'otc_bonds_tplus'}]

In [50]:
# Добавим площадку
for dict_ in url_params:
    engine_id = GLOBAL_GUID.engines.loc[dict_['engine']].id
    slice_market_df = GLOBAL_GUID.markets.loc[dict_['market']]
    if isinstance(slice_market_df, pd.DataFrame):
        market_id = slice_market_df.query("trade_engine_id==@engine_id").iloc[0].market_id
    else:
        market_id = slice_market_df.market_id
    board_group_id = GLOBAL_GUID.boardgroups.loc[dict_["boardgroup"]].id

    dict_['board'] = GLOBAL_GUID.boards.query(
    "board_group_id==@board_group_id and engine_id==@engine_id and market_id==@market_id "
    " and is_primary==1"
    ).sort_values(by=['id']).iloc[0].name
    
url_params

[{'engine': 'stock',
  'market': 'shares',
  'boardgroup': 'stock_shares_tplus',
  'board': 'TQBR'},
 {'engine': 'currency',
  'market': 'selt',
  'boardgroup': 'currency',
  'board': 'CETS'},
 {'engine': 'futures',
  'market': 'forts',
  'boardgroup': 'futures_forts',
  'board': 'RFUD'},
 {'engine': 'state',
  'market': 'bonds',
  'boardgroup': 'state',
  'board': 'MAIN'},
 {'engine': 'commodity',
  'market': 'futures',
  'boardgroup': 'commodity_futures',
  'board': 'FOCM'},
 {'engine': 'interventions',
  'market': 'grain',
  'boardgroup': 'interventions_grain',
  'board': 'GSEL'},
 {'engine': 'offboard',
  'market': 'bonds',
  'boardgroup': 'offboard_bond_all',
  'board': 'OBBO'},
 {'engine': 'agro',
  'market': 'sugar',
  'boardgroup': 'agro_sugar_all',
  'board': 'SUGR'},
 {'engine': 'otc',
  'market': 'bonds',
  'boardgroup': 'otc_bonds_tplus',
  'board': 'OCTR'}]

In [51]:
GLOBAL_GUID.boards.loc[[x["board"] for x in url_params]]

,id,board_group_id,engine_id,market_id,board_title,is_traded,has_candles,is_primary
boardid,,,,,,,,
TQBR,129,57,1,1,Т+: Акции и ДР - безадрес.,1,1,1
CETS,21,13,3,10,Системные сделки - безадрес.,1,1,1
RFUD,101,45,4,22,Фьючерсы,1,1,1
MAIN,19,10,2,6,ГЦБ,0,1,1
FOCM,80,20,5,18,Фьючерсы на товарные активы,0,1,1
GSEL,94,24,6,20,Интервенции по продаже зерна,1,0,1
OBBO,297,141,7,39,ОТС-система: облигации,0,1,1
SUGR,427,271,9,51,Агро: Сахар,1,1,1
OCTR,1022,1021,1012,1013,ОТС: Облигации Т+ - безадресные,1,1,1


In [52]:
GLOBAL_GUID.securitygroups

,id,title,is_hidden
name,,,
stock_index,12,Индексы,0
stock_shares,4,Акции,0
stock_bonds,3,Облигации,0
currency_selt,9,Валюта,0
futures_forts,10,Фьючерсы,0
futures_options,26,Опционы,0
stock_dr,18,Депозитарные расписки,0
stock_foreign_shares,33,Иностранные ц.б.,0
stock_eurobond,6,Еврооблигации,0


In [53]:
# Добавим securitygroup (ручками)
group_entity = {
    "stock": "stock_shares",
    "currency": "currency_selt",
    "futures": "futures_forts",
    "state": "stock_bonds",
    "commodity": "futures_forts",
    "offboard": "stock_bonds",
    "otc": "stock_bonds",
}
for d in url_params:
    if d['engine'] in group_entity:
        d['securitygroup'] = group_entity[d['engine']]
url_params

[{'engine': 'stock',
  'market': 'shares',
  'boardgroup': 'stock_shares_tplus',
  'board': 'TQBR',
  'securitygroup': 'stock_shares'},
 {'engine': 'currency',
  'market': 'selt',
  'boardgroup': 'currency',
  'board': 'CETS',
  'securitygroup': 'currency_selt'},
 {'engine': 'futures',
  'market': 'forts',
  'boardgroup': 'futures_forts',
  'board': 'RFUD',
  'securitygroup': 'futures_forts'},
 {'engine': 'state',
  'market': 'bonds',
  'boardgroup': 'state',
  'board': 'MAIN',
  'securitygroup': 'stock_bonds'},
 {'engine': 'commodity',
  'market': 'futures',
  'boardgroup': 'commodity_futures',
  'board': 'FOCM',
  'securitygroup': 'futures_forts'},
 {'engine': 'interventions',
  'market': 'grain',
  'boardgroup': 'interventions_grain',
  'board': 'GSEL'},
 {'engine': 'offboard',
  'market': 'bonds',
  'boardgroup': 'offboard_bond_all',
  'board': 'OBBO',
  'securitygroup': 'stock_bonds'},
 {'engine': 'agro',
  'market': 'sugar',
  'boardgroup': 'agro_sugar_all',
  'board': 'SUGR'},
 

In [54]:
GLOBAL_GUID.securitygroups.loc['stock_shares']

id               4
title        Акции
is_hidden        0
Name: stock_shares, dtype: object

In [55]:
# collection
set_group = {x['securitygroup'] for x in url_params if x.get('securitygroup')}
display(GLOBAL_GUID.securitygroups.loc[list(set_group)]) #[].query("security_group_id==4")
GLOBAL_GUID.securitycollections.query("security_group_id.isin([10,3,4,9])").sort_values('id')

,id,title,is_hidden
name,,,
currency_selt,9,Валюта,0
stock_bonds,3,Облигации,0
stock_shares,4,Акции,0
futures_forts,10,Фьючерсы,0


,id,title,security_group_id
name,,,
stock_shares_all,3,Все акции,4
stock_bonds_all,7,Все,3
stock_shares_one,160,Уровень 1,4
stock_shares_two,161,Уровень 2,4
stock_shares_three,162,Уровень 3,4
stock_bonds_one,163,Все уровень 1,3
stock_bonds_two,164,Все уровень 2,3
stock_bonds_three,165,Все уровень 3,3
currency_selt_all_swap,170,Все валюты СВОП,9


In [56]:
# Добавим collection (ручками)
group_entity = {
    "stock": "stock_shares_all",
    "currency": "currency_selt_usd_spot",
    "futures": "futures_forts_shares",
    "state": "stock_bonds_ofz_all",
    "offboard": "stock_bonds_all",
    "otc": "stock_bonds_all",
}
for d in url_params:
    if d['engine'] in group_entity:
        d['collection'] = group_entity[d['engine']]
url_params

[{'engine': 'stock',
  'market': 'shares',
  'boardgroup': 'stock_shares_tplus',
  'board': 'TQBR',
  'securitygroup': 'stock_shares',
  'collection': 'stock_shares_all'},
 {'engine': 'currency',
  'market': 'selt',
  'boardgroup': 'currency',
  'board': 'CETS',
  'securitygroup': 'currency_selt',
  'collection': 'currency_selt_usd_spot'},
 {'engine': 'futures',
  'market': 'forts',
  'boardgroup': 'futures_forts',
  'board': 'RFUD',
  'securitygroup': 'futures_forts',
  'collection': 'futures_forts_shares'},
 {'engine': 'state',
  'market': 'bonds',
  'boardgroup': 'state',
  'board': 'MAIN',
  'securitygroup': 'stock_bonds',
  'collection': 'stock_bonds_ofz_all'},
 {'engine': 'commodity',
  'market': 'futures',
  'boardgroup': 'commodity_futures',
  'board': 'FOCM',
  'securitygroup': 'futures_forts'},
 {'engine': 'interventions',
  'market': 'grain',
  'boardgroup': 'interventions_grain',
  'board': 'GSEL'},
 {'engine': 'offboard',
  'market': 'bonds',
  'boardgroup': 'offboard_bond

In [57]:
ENTITY_DESCRIPTIONS

{'asset': 'Secid базового актива для опционов.',
 'board': 'Код торговой площадки (режим торгов)',
 'boardgroup': 'Группа торговых площадок',
 'engine': 'Признак рынка (фондовый, валютный, срочный...)',
 'market': 'Классфикация рынка: спот, индексы, рпс, репо, размещение, валюта и т.п.',
 'securitygroup': 'Класс финансового инструмента',
 'collection': 'Группировка финансовых инструментов по признакам (уровень, тип и тд). В словаре используется как securitycollection',
 'datatype': 'securities or trades',
 'event_id': 'ID мероприятия проводимого Мосбиржей. Идут в порядке возрастания. Только ближайшие события можно посмотреть на /iss/events',
 'indexid': 'Код индекса (бенчмарка) публикуемого и/или расчитываемого Мосбиржей',
 'news_id': 'Код новостной рассылки Мосбиржи. Реестр новостей можно увидить на /iss/sitenews',
 'period': 'yearly, monthly или daily',
 'report_name': 'Еженедельные отчеты по ВАЛЮТНЫМ (! только !) деривативам. По первичному анализу - OTC / c ЦК и нестандартные ПФИ. Н

In [58]:
# Дефолтные значения:
for d in url_params:
    d["datatype"] = "securities"
    d["event_id"] = 10500
    d["news_id"] = 30000
    d["period"] = "monthly"
    d["session"] = 3
    d["year"] = 2022
    d["ticker"] = 'GAZP'
    d["indexid"] = "RTSI"
url_params[2]['report_name'] = "openpositions"
url_params[2]['asset'] = "GAZP"

In [59]:
# Остался SECID:
url = (
    "http://iss.moex.com/iss/engines/{engine}/markets/{market}/boards/{board}/securities.json?iss.meta=off"
    "&iss.only=securities&securities.columns=SECID,SHORTNAME,STATUS,SECTYPE,LISTLEVEL&securities.sort_column=LISTLEVEL"
)
res_dict = {}
for d in url_params:
    res_dict[d["engine"]] = requests.get(url.format(**d)).json()
    time.sleep(.25)

In [60]:
print([x['engine'] for x in url_params])
secid_engine = {
    "stock": "GAZP",
    "currency": "USDRUB_TMS",
    "futures": "GZU5",
    "state": "SU46023RMFS6",
    "commodity": "FDEXW1RGG135",
    "offboard": "RU000A103D03",
    "agro": "SP_SG_PERL",
    "otc": "RU000A0ZYYN4"
}
url_params[8]['board'] = 'OCBR'
url_params[8]['market'] = 'ndm'
url_params[8]['boardgroup'] = "otc_ndm_ccp_rub"

['stock', 'currency', 'futures', 'state', 'commodity', 'interventions', 'offboard', 'agro', 'otc']


In [61]:
for url in url_params:
    if url['engine'] in secid_engine:
        url['security'] = secid_engine[url['engine']]
url_params

[{'engine': 'stock',
  'market': 'shares',
  'boardgroup': 'stock_shares_tplus',
  'board': 'TQBR',
  'securitygroup': 'stock_shares',
  'collection': 'stock_shares_all',
  'datatype': 'securities',
  'event_id': 10500,
  'news_id': 30000,
  'period': 'monthly',
  'session': 3,
  'year': 2022,
  'ticker': 'GAZP',
  'indexid': 'RTSI',
  'security': 'GAZP'},
 {'engine': 'currency',
  'market': 'selt',
  'boardgroup': 'currency',
  'board': 'CETS',
  'securitygroup': 'currency_selt',
  'collection': 'currency_selt_usd_spot',
  'datatype': 'securities',
  'event_id': 10500,
  'news_id': 30000,
  'period': 'monthly',
  'session': 3,
  'year': 2022,
  'ticker': 'GAZP',
  'indexid': 'RTSI',
  'security': 'USDRUB_TMS'},
 {'engine': 'futures',
  'market': 'forts',
  'boardgroup': 'futures_forts',
  'board': 'RFUD',
  'securitygroup': 'futures_forts',
  'collection': 'futures_forts_shares',
  'datatype': 'securities',
  'event_id': 10500,
  'news_id': 30000,
  'period': 'monthly',
  'session': 3

### Первичный список параметров запроса готов. При необходимости скорректируем его

План проверки эндпоинтов следующий
1. Отсеить все поинты, доступные только для подписчиков
2. Проверить, что поинты что-то получают.
    - Если поинт что-то получил - сохранить результат, отметить результат как успешный
    - Если поинт получил не 200 - отметить поинты с ошибкой
    - Если поинт получил ошибку в данных (есть ключ с именем "error") - добавить поинт в проблемный
3. Проверить результаты, повторить

In [77]:
# Заведем словари для хранения, где ключ - id ссылки на описание API
OK_API = {}
BAD_API = {}
UNAVAILABLE_API = {}
ERROR_API = {}
EMPTY_RESP_API = {}
NOT_DATA_API = {}

save_path = 'moex_api_response'

if not os.path.exists(save_path):
    os.mkdir(save_path)

In [111]:
# Пройдемся по все урлам без параметров:

def find_error(json_res: dict):
    for key, value in json_res.items():
        if key == 'error' or (isinstance(value, dict) and find_error(value)):
            return True
    return False


def check_null(json_res: dict):
    for key, value in json_res.items():
        if key == "data":
            return (True, True) if value else (True, False)
        if isinstance(value, dict): 
            is_find, res = check_null(value)
            if is_find:
                return is_find, res
    return False, None


def save_result(json_res: dict, api_id):
    with open(save_path + '/' + str(api_id) + '.json', 'w') as outfile:
        json.dump(json_res, outfile)
        
def check_url_in_result(api_id: int):
    return any(api_id in dict_ for dict_ in [OK_API, BAD_API, UNAVAILABLE_API, ERROR_API, EMPTY_RESP_API, NOT_DATA_API])

def check_api(url_data: MoexApi):
    api_id = int(re.findall(r"\d+$", url_data.url_faq)[-1])
    if check_url_in_result(api_id):
        return 
    
    try:
        resp = requests.get(url_data.url_template + '.json')
        resp_json = resp.json()
    except:
        UNAVAILABLE_API[api_id] = url_data
        return
        
    if resp.status_code != 200:
        BAD_API[api_id] = [url_data, resp_json]
        return

    if find_error(resp_json):
        ERROR_API[api_id] = [url_data, resp_json]
        return
    
    is_find, res = check_null(resp_json)
    if not is_find:
        NOT_DATA_API[api_id] = [url_data, resp_json]
    elif not res:
        EMPTY_RESP_API[api_id] = [url_data, resp_json]
    else:
        OK_API[api_id] = url_data
        
    save_result(resp_json, api_id)
        
    
for url_data in FINAL_DATA:
    if url_data.url_template.find('{') == -1:
        check_api(url_data)
        time.sleep(.25)

In [79]:
print(BAD_API, ERROR_API, EMPTY_RESP_API, NOT_DATA_API)

{} {} {} {}


In [80]:
print(list(OK_API.values())[:5])
print("Всего:", len(OK_API))

[MoexApi(url_faq='http://iss.moex.com/iss/reference/5', url_template='http://iss.moex.com/iss/securities', description='Список бумаг торгуемых на московской бирже.'), MoexApi(url_faq='http://iss.moex.com/iss/reference/24', url_template='http://iss.moex.com/iss/turnovers', description='Получить сводные обороты по рынкам.Например: https://iss.moex.com/iss/turnovers.xml'), MoexApi(url_faq='http://iss.moex.com/iss/reference/100', url_template='http://iss.moex.com/iss/turnovers/columns', description='Получить описание полей для запросов оборотов по рынку/торговой системе.\rНапример: https://iss.moex.com/iss/engines/stock/turnovers/columns.xml'), MoexApi(url_faq='http://iss.moex.com/iss/reference/28', url_template='http://iss.moex.com/iss/index', description='Получить глобальные справочники ISS. \rНапример: https://iss.moex.com/iss/index.xml'), MoexApi(url_faq='http://iss.moex.com/iss/reference/40', url_template='http://iss.moex.com/iss/engines', description='Получить доступные торговые сист

In [81]:
print(UNAVAILABLE_API)  # Прочекал доп. глазами
print("Всего:", len(UNAVAILABLE_API))

{833: MoexApi(url_faq='http://iss.moex.com/iss/reference/833', url_template='http://iss.moex.com/iss/history/otc/providers/nsd/markets', description='Обобщенные данные ОТС ПФИ и РЕПО - список рынков.'), 841: MoexApi(url_faq='http://iss.moex.com/iss/reference/841', url_template='http://iss.moex.com/iss/statistics/engines/stock/securitieslisting', description='Таблица соответствия торгуемых ценных бумаг по режимам торгов'), 905: MoexApi(url_faq='http://iss.moex.com/iss/reference/905', url_template='http://iss.moex.com/iss/statistics/complex/securities', description='Маркировка сложных финансовых инструментов')}
Всего: 3


#### Теперь прочекаем урлы с вшитыми параметрами
Логика как и без них, но если подставить в УРЛ недопустимые параметры (например, урл только для индексов, а подставляем цб) то можем получить ошибку или пустые данные. В этом случае нам нужно взять следующий массив из готовых параметров и протестить на них, пока не получим нужные нам данные.
Для дальнейшего анализа и оптимизации, сохраним engine набора параметров для успешных запросов.
Пустому ответу отадаем приоритет выше, чем ответу с ошибкой (+ сохраним engine набора, по которому вернули пустые данные).

In [98]:
OK_ENGINE = {}
EMPTY_ENGINE = {}

In [99]:
# url_params
def check_api_with_params(url_data: MoexApi):
    api_id = int(re.findall(r"\d+$", url_data.url_faq)[-1])
    if check_url_in_result(api_id):
        return 
    
    tmp_error_json = None
    tmp_empty_json = None
    tmp_engine = None
    
    for params in url_params:
        
        try:
            use_url = url_data.url_template.format(**params)
        except KeyError:
            continue
        
        try:
            time.sleep(.25)
            resp = requests.get(use_url + '.json')
            resp_json = resp.json()
        except json.JSONDecodeError:
            UNAVAILABLE_API[api_id] = url_data
            return
        
        if resp.status_code != 200:
            BAD_API[api_id] = [url_data, resp_json]
            return

        if find_error(resp_json):
            if tmp_empty_json is None and tmp_error_json is None:
                tmp_error_json = resp_json
                tmp_engine = params['engine']
            continue

        is_find, res = check_null(resp_json)
        if not is_find:
            NOT_DATA_API[api_id] = [url_data, resp_json]
        elif not res:
            if tmp_empty_json is None:
                tmp_empty_json = resp_json
                tmp_engine = params['engine']
            continue
        else:
            OK_API[api_id] = url_data
            OK_ENGINE[api_id] = params['engine']
            save_result(resp_json, api_id)
            return
    
    if tmp_engine is None:
        print("For urls", url_data, "not found params")
    elif tmp_empty_json:
        EMPTY_RESP_API[api_id] = [url_data, tmp_empty_json]
        EMPTY_ENGINE[api_id] = tmp_engine
        save_result(tmp_empty_json, api_id)
    else:
        ERROR_API[api_id] = [url_data, tmp_error_json]
        save_result(tmp_error_json, api_id)
        

In [100]:
for url_data in FINAL_DATA:
    if url_data.url_template.find('{') != -1:
        check_api_with_params(url_data)

For urls MoexApi(url_faq='http://iss.moex.com/iss/reference/789', url_template='http://iss.moex.com/iss/history/engines/{engine}/markets/{market}/.*?{securities}/columns', description='Получить описание полей для запросов исторических данных по бумагам для рынка.') not found params


In [101]:
EMPTY_RESP_API

{158: [MoexApi(url_faq='http://iss.moex.com/iss/reference/158', url_template='http://iss.moex.com/iss/engines/{engine}/markets/{market}/boardgroups/{boardgroup}/securities/{security}/candleborders', description=''),
  {'borders': {'metadata': {'begin': {'type': 'datetime',
      'bytes': 57,
      'max_size': 0},
     'end': {'type': 'datetime', 'bytes': 57, 'max_size': 0},
     'interval': {'type': 'int32'},
     'board_group_id': {'type': 'int32'}},
    'columns': ['begin', 'end', 'interval', 'board_group_id'],
    'data': []}}],
 759: [MoexApi(url_faq='http://iss.moex.com/iss/reference/759', url_template='http://iss.moex.com/iss/statistics/engines/stock/splits/{security}', description=''),
  {'splits': {'metadata': {'tradedate': {'type': 'date',
      'bytes': 10,
      'max_size': 0},
     'secid': {'type': 'string', 'bytes': 36, 'max_size': 0},
     'before': {'type': 'int32'},
     'after': {'type': 'int32'}},
    'columns': ['tradedate', 'secid', 'before', 'after'],
    'data': 

In [102]:
len(OK_API)

104

In [87]:
UNAVAILABLE_API

{}

### В целом, недоступные урлы найдены корректно. 
Описание полей нас не особо интересует. Часть аналитики и заявки - недоступны без подписки!

### API c ошибками:

In [103]:
ERROR_API

{120: [MoexApi(url_faq='http://iss.moex.com/iss/reference/120', url_template='http://iss.moex.com/iss/history/engines/{engine}/markets/{market}/boardgroups/{boardgroup}/listing', description='Получить данные по листингу бумаг в историческом разрезе по указанной группе режимов'),
  {'securities': {'error': 'Error while acquiring data.'}}],
 219: [MoexApi(url_faq='http://iss.moex.com/iss/reference/219', url_template='http://iss.moex.com/iss/statistics/engines/{engine}/derivatives/{report_name}', description='Еженедельные отчеты по валютным деривативам:\r\rnumtrades - Информация о количестве договоров по инструментам, являющимся производными финансовыми инструментами (по валютным парам)\rparticipants - Информация о количестве лиц, имеющих открытые позиции по инструментам, являющимся производными финансовыми инструментами (по валютным парам)\ropenpositions - Информация об открытых позициях по инструментам, являющимся производными финансовыми инструментами (по валютным парам)\rexpirationpar

1. Листинг в историческом разрезе нас не интересует.
2. Отчеты по валютных деривативам выдают результат, при выборе engine='currency'. Но без деталей - поинт бесполезный (либо пустой результат либо объемы несопоставимы с оборотом срочки)

In [104]:
# ТОП! ВСЕ ОТВЕТЫ СТАНДАРТНОГО ВИДА!!!
NOT_DATA_API 

{}

In [105]:
display(EMPTY_RESP_API)
print(len(EMPTY_RESP_API))

{158: [MoexApi(url_faq='http://iss.moex.com/iss/reference/158', url_template='http://iss.moex.com/iss/engines/{engine}/markets/{market}/boardgroups/{boardgroup}/securities/{security}/candleborders', description=''),
  {'borders': {'metadata': {'begin': {'type': 'datetime',
      'bytes': 57,
      'max_size': 0},
     'end': {'type': 'datetime', 'bytes': 57, 'max_size': 0},
     'interval': {'type': 'int32'},
     'board_group_id': {'type': 'int32'}},
    'columns': ['begin', 'end', 'interval', 'board_group_id'],
    'data': []}}],
 759: [MoexApi(url_faq='http://iss.moex.com/iss/reference/759', url_template='http://iss.moex.com/iss/statistics/engines/stock/splits/{security}', description=''),
  {'splits': {'metadata': {'tradedate': {'type': 'date',
      'bytes': 10,
      'max_size': 0},
     'secid': {'type': 'string', 'bytes': 36, 'max_size': 0},
     'before': {'type': 'int32'},
     'after': {'type': 'int32'}},
    'columns': ['tradedate', 'secid', 'before', 'after'],
    'data': 

8


### Пройдемся по id-шникам

- 158. Нет описания
- 759. Нет описания
- 149. Играл руками. НЕ получается вытянуть показатели и статистику бумаги участвующей в индексе. Возможно нужен перебор (возможно, работает только с какими-то опредленными индексами)
- 163. Играл с кучей кодов режимов. Всегда возвращет пустоту. Невозможно получить обобщенную инфу.
- 164. Аналогично с 163, но с деталями по бумаге. Пустота
- 715. Нужен код фиксинга и заработает. Исправляю ниже.
- 712. Нужен код индикативного курса валют. Похоже secid ~ USD/RUR (т.е. через знак черты '/'). Исправляю ниже.
- 861. Нужен код ПФИ. Исправлю ниже

In [106]:
data = EMPTY_RESP_API.pop(715)
resp = requests.get(data[0].url_template.format(security="EURFIXME") + '.json')
save_result(resp.json(), 715)
OK_API[715] = data[0]

In [107]:
data = EMPTY_RESP_API.pop(712)
resp = requests.get(data[0].url_template.format(security="USD/RUB") + '.json')
save_result(resp.json(), 712)
OK_API[712] = data[0]

In [108]:
data = EMPTY_RESP_API.pop(861)
resp = requests.get(data[0].url_template.format(security="ALRS-6.23") + '.json')
save_result(resp.json(), 861)
OK_API[861] = data[0]

# Финалочка:


In [112]:
all_api = len(FINAL_DATA)
bad_api = len(UNAVAILABLE_API) + len(EMPTY_RESP_API) + len(ERROR_API) + len(BAD_API)
good_api = len(OK_API)
print(all_api, bad_api, good_api)  # Потеряли api #789 т.к. там не подходит параметр. Эндпоинт с описанием полей.

151 43 107


In [ ]:
# Большая часть остатка - по подписке или не работают. Малая часть - описание полей, которую можно пропустить.
print(f"Проверено {good_api} из {all_api} эндпоинтов. Или {good_api/all_api:.0%}.")

## Итого:
1. Мы получили список рабочих урлов и потенциально можем написать решение для быстрого и удобного их использования
2. У нас есть большАя часть описания урлов.
3. У нас есть примеры ответов, для быстрого изучения деталей.

## Планы:

1. Собрать описание детальных параметров.
2. Проверить их работоспособность.
3. Подготовить инструмент быстрой работы с апи моекса

In [116]:
# Save for next iteration
import pickle
save_pickle = {
    "use_api": OK_API,
    "globals": GLOBAL_GUID
}
with open('api.pickle', 'wb') as file:
    pickle.dump(save_pickle, file)